<a href="https://colab.research.google.com/github/Rajeeb321123/Deep-learning-tensorFlow-Journey/blob/master/08_introduction_to_NLP_in_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to NLP Fundamental in TensorFLow

NLP has the goal of deriving out of natural language ( could be sequence text or speech)

Another  common term for NLP problems is sequence problem (seq2seq).

## Check GPU

In [10]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


## Get the helper function

In [11]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

# Import series of helper function for the notebook

from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

--2024-04-04 13:32:57--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2024-04-04 13:32:57 (103 MB/s) - ‘helper_functions.py’ saved [10246/10246]



## Get a text dataset

The dataset we're going to be using is Kaggle's introduction to NLP dataset (text samples of Tweets labelled as disaster or not disaster).

original source of dataset: https://www.kaggle.com/competitions/nlp-getting-started

In [12]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

# Unzip the data
unzip_data("nlp_getting_started.zip")

--2024-04-04 13:32:58--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.107.207, 74.125.196.207, 173.194.218.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.107.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.005s  

2024-04-04 13:32:58 (108 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



## Visualizing a text dataset

To Visualize our text samples, we first have to read them in, one way to do so would be to use Python: https://realpython.com/read-write-files-python/

But I perfer get visual straight away.

So anther way to do this use pandas... (may not work for large dataset exceeding RAM: here 16 GB).

for loading large text dataest: https://www.tensorflow.org/tutorials/load_data/text


In [13]:
import pandas as pd
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.head(n=10)

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
5,8,NaN,NaN,#RockyFire Update => California Hwy. 20 closed...,1
6,10,NaN,NaN,#flood #disaster Heavy rain causes flash flood...,1
7,13,NaN,NaN,I'm on top of the hill and I can see a fire in...,1
8,14,NaN,NaN,There's an emergency evacuation happening now ...,1
9,15,NaN,NaN,I'm afraid that the tornado is coming to our a...,1


In [14]:
train_df["text"][10]

'Three people died from the heat wave so far'

In [15]:
# Shuffle training dataframe
train_df_shuffled = train_df.sample(frac=1, random_state=42) # 1 means 100% of data, 0.1 means 10% of data
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [16]:
# What does the test dataframe look like?
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [17]:
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [18]:
# How many example s of each class are there?
# for knowing our dataset is balanced or not?
train_df.target.value_counts()

#60 and 40 split between 0 and 1 : pretty balance (50-50)

target
0    4342
1    3271
Name: count, dtype: int64

Here we work with almost balance dataset but for imbalance dataset ( no near 50-50 split) refer to: https://www.tensorflow.org/tutorials/structured_data/imbalanced_data

In [19]:
# How many total samples?
len(train_df), len(test_df)

(7613, 3263)

In [20]:
a = train_df_shuffled["target"] # 1d
print(a.values, a.values.shape) # array([1, 0, 1, ..., 0, 1, 1])

a = train_df_shuffled[["target"]] # 2d
print(a.values, a.values.shape) # array([[1], [0], [1], ..., [1]])

a = train_df_shuffled[["text","target"]] # 2d
print(a.values[0], a.values.shape)

[1 0 1 ... 0 1 1] (7613,)
[[1]
 [0]
 [1]
 ...
 [0]
 [1]
 [1]] (7613, 1)
['So you have a new weapon that can cause un-imaginable destruction.' 1] (7613, 2)


In [21]:
# a little tutorial on comprehension
a = [1, 2, 3, 4, 5]

b = [ x if x < 3 else "" for x in a]
print(b)
b = [ x  for x in a if x < 3]
print(b)

[1, 2, '', '', '']
[1, 2]


In [22]:
# Let's visualize some random training dataset
import random
random_index = random.randint(0, len(train_df)-5) # create a random index not higher than total no of samples
print(f"random_index: {random_index}")
#itertuples turing dataframe values to tuple
for row in train_df_shuffled[["text", "target"]][random_index:random_index+5].itertuples(): # random index eg 170 to 175, five values
  _, text, target = row # skipping index
  print(f"Target: {target}", "(real disaster)" if target > 0 else "{not real disaster}")
  print(f"Text:\n{text}\n")
  print("---\n")

random_index: 2599
Target: 0 {not real disaster}
Text:
Ladies here's how to recover from a #date you totally BOMBED... according to men http://t.co/c5GGSZUGw1 http://t.co/2PiMg9BIcE

---

Target: 0 {not real disaster}
Text:
@Glosblue66 no idea what this means. Look at our violent crime rate without weapons. Ban guns we become like Mexico not Australia

---

Target: 1 (real disaster)
Text:
ÛÏ@based_georgie: yo forreal we need to have like an emergency action plan incase donald trump becomes presidentÛ
Whipe that lil baby

---

Target: 0 {not real disaster}
Text:
[CLIP] Top-down coercion - The structural weakness ensuring government failure http://t.co/gNORIjnSVa

---

Target: 0 {not real disaster}
Text:
Listen to Landslide by Oh Wonder #SoundCloud https://t.co/SJkgJxff2r

---



### Split data into training and validation sets

In [23]:
from sklearn.model_selection import train_test_split

In [24]:
# Use train_test_split to split  to split training and validation set

train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size=0.1, # use 10% of training data for validation,
                                                                            random_state=42)

In [25]:
# Check the lengths
len(train_sentences),len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [26]:
# Check the first 10 samples
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

## Converting text to numbers

* Tokenization vs Embedding video no: 228 watch full video -> must

When dealing with a text problem, one of the first things we'll have to do before we can build a model is to convert our text to numbers.

There are a few ways to do this, namely:
* Tokenization - direct mapping of token (a token could be a word or a character) to number.
* `Embedding` - create a matrix of feature vector fo each each token (the size of feature vector can be defined and `weights can be learned`).

In [1]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

In [9]:
# use the default TextVectorization parameters
# This is only the default bu we will modify later here.
#max_token=10000 means only 10000 most common word
text_vectorizer = TextVectorization(max_tokens=None, # How many words in our vocabulary: None: no limit ( automaically add <00V>)
                                    standardize = 'lower_and_strip_punctuation',
                                    split='whitespace',
                                    ngrams=None, # n-words:join splitted words before tokenizaion. None:each word, each token
                                    output_mode='int', # how to map token to token. here int but can be specific function like tf_idf
                                    #How long do we want our sequence to be : 50 ,100, None..
                                    #automatially set sequence to longest sequence or tweet lenght. add zero ad end for short tweets
                                    output_sequence_length=None, # Batch need to same length. Eg here Each Tweet length are different
                                    # aadd zero ad end for short tweets to match (output_sequence_length: none: means longest one)
                                    pad_to_max_tokens=False #if true reult in shape (Batch_size, Max_tokesn)
                                    )

In [27]:
train_sentences[0].split()

['@mogacola', '@zamtriossu', 'i', 'screamed', 'after', 'hitting', 'tweet']

In [31]:
len(train_sentences[0].split())

7

In [32]:
# Find the average number of tokens (words) in the training tweets for keeping our model small
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

# seem no of token or words is 15

15

In [51]:
# Setup text Vectorization variable
max_vocab_length = 10000 # max number of words  to have in our vocabulary
# 15 from round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))
max_length= 15 # max length our sequence will be (e.g. how many words from a tweet does a model see?)

text_vectorizer = TextVectorization(max_tokens = max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length = max_length,
                                    pad_to_max_tokens=True,
                                    )

In [53]:
# Fit the text vectorizer to the training text
text_vectorizer.adapt(train_sentences)

In [54]:
# Create a sample sentences and tokenize it
sample_sentence = "There's a flood in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [55]:
# Choose  a random sentence from the training dataset and tokenize it
random_sentence = random.choice(train_sentences)
print(f"Original text:\n {random_sentence}\
      \n\nVectorized version:")
text_vectorizer([random_sentence])

Original text:
 Goulburn man Henry Van Bilsen missing: Emergency services are searching for a Goulburn man who disappeared from hisÛ_ http://t.co/z99pKJzTRp      

Vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[5549,   89, 5489, 1929,    1,  373,   73,  327,   22,  669,   10,
           3, 5549,   89,   65]])>

In [56]:
# How unique words from vocabulary we found in our train_sentence
# if Max_token: None as hyperparameter : automatically set no of words_in_vocab to each Unique word
# if Max_token: 10000 like we did : words_in_vocab will be 100000
words_in_vocab = text_vectorizer.get_vocabulary() # get all of the unique words in our train_data
top_5_words = words_in_vocab[:5] # get the most common words
bottom_5_words = words_in_vocab[-5:] # get the least common words

print(f"Number of words in vocab in train_data: {len(words_in_vocab)}")
print(f"5 most common words: {top_5_words}")
print(f"5 least common words: {bottom_5_words}")

#[UNK] means unkown: 10000 words may be small but its ok here

Number of words in vocab in train_data: 10000
5 most common words: ['', '[UNK]', 'the', 'a', 'in']
5 least common words: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


### Creating a Embedding using an Embedding layer

To make our embedding, we're going to use TensorFlow's embeddings layer: https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding

The parameter we care most about for our embedding layer:
* `input_dim` = the size of our vocabulary.
* `output_dim` = the size our output embedding vector, for example, a value of 100 would mean each token get represented by a vector 100 long.
* `input_length` = length of the sequences being passed to the embedding layer.


In [60]:
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim = max_vocab_length, # set the input shape,
                              output_dim = 128, # Neural network especially that using GPU works greate with number divisible by 8 eg. 128
                              input_length = max_length, # how long is each input
                              )
embedding

In [66]:
# for visualizaion only on random sentence

# Get a random sentence from the training set
random_sentence = random.choice(train_sentences)
print(f"Original text:\n {random_sentence}\
      \n\nEmbedded version:")

# Embed the random sentence (turn it into dense vector of fixed size)
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
 Worked in factory pressing designs onto T-shirts was electrocuted 
d/t faulty ground. Boss docked my pay while I was at ER #WorstSummerJob      

Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.03632266, -0.00822704,  0.01014316, ..., -0.01248065,
         -0.03385881,  0.03897716],
        [ 0.03911574, -0.03925375, -0.01700457, ...,  0.00368136,
          0.03313467, -0.02994938],
        [ 0.02500122, -0.02322021, -0.01500287, ..., -0.01850743,
          0.02211369,  0.0034605 ],
        ...,
        [ 0.02381768, -0.01589816, -0.03759276, ..., -0.03341293,
         -0.02554506, -0.01940012],
        [ 0.02283089, -0.04068989, -0.00266194, ..., -0.03455757,
         -0.02374883, -0.01433007],
        [-0.03842049, -0.0225729 , -0.03770094, ...,  0.00051112,
         -0.03216507,  0.01444593]]], dtype=float32)>

In [67]:
# Check out a single token's embedding for the first word of our sentence e.g. Worked
sample_embed[0][0], sample_embed[0][0].shape, random_sentence

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([ 0.03632266, -0.00822704,  0.01014316,  0.04242152, -0.04335816,
         0.01407281, -0.03734419,  0.02595513, -0.03604578,  0.01637124,
         0.02584234, -0.04796841, -0.03154287, -0.01427569,  0.00251721,
         0.02356149,  0.02058866, -0.04109647,  0.04862925,  0.00630096,
         0.04449658, -0.02829623,  0.0439438 , -0.03662727, -0.02061613,
        -0.02346612, -0.01650698, -0.03425767,  0.00749902, -0.03093444,
         0.00015998,  0.03660226,  0.03642792, -0.02789406, -0.02895676,
         0.02371717, -0.00526337,  0.03832033,  0.00826523,  0.03115192,
         0.04244919, -0.02666196, -0.01592273, -0.01627594, -0.02209611,
         0.00255651, -0.00916886,  0.04550702, -0.03201829,  0.00619126,
         0.03670298,  0.01581972, -0.04196712, -0.01458244,  0.04583284,
         0.02233917,  0.02212561, -0.01849947, -0.00563629,  0.0067733 ,
        -0.04073987, -0.04648552, -0.00540547,  0.00350922,  0.04476726,
  